In [1]:
projectbase = 'F:\\ANU\\Research\\Polyanya\\anyangle\\polyanya\\'
testMapfolder = 'meshes\\'
weightedFolder = 'weightedMeshes\\'
#set the squarePath locally.
squarePath="F:\\ANU\\Research\\Polyanya\\anyangle\\polyanya\\weightedMeshes\\tests\\bad-collinear.mesh"
import decimal
from decimal import *
decimal.getcontext().rounding = "ROUND_HALF_UP"

approximate = 1/10000


In [2]:
import os
import random
def giveWeights(projectbase, testMapfolder,weightedFolder):
    for curDir, dirs, files in os.walk(projectbase+testMapfolder):
        for filename in files:
            if '.mesh' in filename:   
                r_path = curDir+'\\'+filename
                w_path = r_path.replace(testMapfolder,weightedFolder)
                giveWeight(r_path,w_path)
                
def giveWeight(r_path,w_path):
                print(r_path)
                with open(r_path,'r',encoding = 'utf-8') as f:
                    #write
                    if os.path.exists(w_path):
                        os.remove(w_path)
                    file_write_obj = open(w_path, 'w')
                
                    #read
                    lines = f.readlines()
                    n_vertices = 0
                    n_poly = 0
                    count_ver = 0
                    count_poly = 0
                    for i in range(0,len(lines)):
                        w_line = lines[i].strip()
                        if i == 0:
                            w_line = "weighted_"+w_line
                        if i == 2:
                            s_vp = w_line.split(" ")
                            n_vertices = int(s_vp[0])
                            n_poly = int(s_vp[1])
                        if count_ver<n_vertices and i>2 and len(w_line)!=0:
                            count_ver+=1
                            
                        if count_ver == n_vertices and count_poly<n_poly and len(w_line)!=0:
                            weight = str(random.randint(1,9))
                            w_line = w_line +" "+ weight
                            
                        if len(w_line)!=0:
                            file_write_obj.writelines(w_line)
                            file_write_obj.write('\n')
                    file_write_obj.close()

In [3]:
#giveWeights(projectbase, testMapfolder,weightedFolder)

In [5]:
#test if the snellPath correct
def SnellPathTest(inciVector,outVector, weightRate,NormalVector):
    #weightRate = weight1/weight2
    cos1 = np.dot(inciVector,NormalVector)
    cos2 = np.dot(outVector,NormalVector)
    print("NormalVector",NormalVector)
    sin1 = (1-cos1**2)**0.5
    sin2 = (1-cos2**2)**0.5
    print("weightRate:",weightRate)
    print("sin2/sin1:",sin2/sin1)
    if weightRate == (sin2/sin1):
        return True
    else:
        return False

In [6]:
#test if a ray hit an edge
#start:startPoing[x1,y1], vector:direction of ray[x1,y1], edge:[[x1,y1],[x2,y2]]
def isInterset(start,vector,edge):
    #print("edge",edge)
    #print("start",start)
    #print("vector",vector)      
    if (start==edge[0]).all():
        return False
    if (start==edge[1]).all():
        return False
    side1 = edge[0]-start
    side2 = edge[1]-start
    side1 = side1/np.linalg.norm(side1)
    side2 = side2/np.linalg.norm(side2)
    tmp1 = np.dot(vector,side1)
    tmp2 = np.dot(vector,side2)
    #print("tmp1",tmp1)
    #print("tmp2",tmp2)
    if (tmp1+tmp2)<=0:
        return False
    result = np.cross(vector,side1)*np.cross(vector,side2)
    if result>=0:
        return False
    else:
        #print("----")
        #print("interset!")
        #print("start:",start)
        #print("vector:",vector)
        #print("edge:",edge)
        return True
# get the IntersetPoint between a ray and an edge
#start:startPoing[x1,y1], vector:direction of ray[x1,y1], edge:[[x1,y1],[x2,y2]]
# returning: Intersection[x1,y1]
def getIntersetPoint(start,vector,edge):
    #print("gettingIntersetPoint")
    #print("")
    v_edge = edge[1]-edge[0]
    p_edge = edge[0]
    start = start
    vector =vector
    startpointSeg = p_edge-start 
    vecS1 = np.cross(vector,v_edge)
    vecS2 = np.cross(v_edge,startpointSeg)
    num2 = -np.dot(vecS2, vecS1) / (np.linalg.norm(vecS1)**2)
    intersection = start + vector * num2
    #print("intersection:",intersection)
    return intersection
       

In [7]:
class Vertex:
    def __init__(self):
        self.x = -1
        self.y = -1
        self.neighbors = []
        self.index=-1
    def getPoint(self):
        return np.array([self.x,self.y])
        
class Polygon:
    def __init__(self):
        self.vertices = []
        self.neighbors = []  #neighbors[i] share vertices[i] and vertices[i+1] with self
        self.weight = 0
        self.min_x = float("inf")
        self.min_y = float("inf")
        self.max_x = -float("inf")
        self.max_y = -float("inf")
        self.Mesh = None
        self.index = -1
    #find if a ray interset with this poly. if it is, returning the intersetpoint and the index of neighbor where the ray is going to pass
    #self:Polygon, start:[x1,y1], vector:[x1,y1], meshObj: Mesh
    def findIncidence(self,start,vector,meshObj):
        numOfEdge = len(self.vertices)
        for i in range(0,numOfEdge):
            l = []
            l.append(self.vertices[i])
            l.append(self.vertices[(i+1)%numOfEdge])
            edge = meshObj.getPoints(l)
            if isInterset(start,vector,edge):
                #print("interset!")
                #print("edge ",edge)
                #print("start ",start)
                #print("vector ",vector)
                #print("neighbors:",i)
                return getIntersetPoint(start,vector,edge),i
        return -1,-1
    # find which edge having this point, return the edge index if it has, raise Exception if it is not here.
    def findEdgebyPoint(self,point,meshObj):
        numOfEdge = len(self.vertices)
        for i in range(0,numOfEdge):
            l = []
            l.append(self.vertices[i])
            l.append(self.vertices[(i+1)%numOfEdge])
            edge = meshObj.getPoints(l)
            #print("point",point)
            #print("edge1,",meshObj.vertices[self.vertices[i]].getPoint())
            #print("edge2,",meshObj.vertices[self.vertices[i-1]].getPoint())
            side1 = np.array(point)-meshObj.vertices[self.vertices[i]].getPoint()
            side2 = np.array(point)-meshObj.vertices[self.vertices[i-1]].getPoint()
            side1/=np.linalg.norm(side1)
            side2/=np.linalg.norm(side2)
            #cross = np.cross(side1,side2)
            dot = np.dot(side1,side2)
            #print("side1,",side1)
            #print("side2,",side2)
            #print(side1)
            #print(side2)
            #print("cross:",cross)
            #print("dot",dot)
            if abs(dot+1)<0.00000000001:
                #print("dot",dot)
                #print("Here!")
                return i-1
        #print("notHere")
        return -1
    #find which edge interset with the ray
    def findInciEdge(self,start,vector,meshObj):
        intersetPoint,neighborIndex = self.findIncidence(start,vector,meshObj)
        edge = self.getEdge(neighborIndex)
        return meshObj.getPoints(edge)
    #find if the poly having this point
    #self:Polygon, point:[x1,y1], vertices: meshObj.vertices
    def isHavingPoint(self,point,vertices):
        if point[0]>self.max_x:
            return False
        if point[1]>self.max_y:
            return False
        if point[0]<self.min_x:
            return False
        if point[1]<self.min_y:
            return False
        start = np.array(point)
        #print("start",start)
        for i in range(0,len(self.vertices)):
            #on vertex
            if (start ==vertices[self.vertices[i]].getPoint()).all():
                #print("on vertex")
                return True
            #on edge
            side1 = start-vertices[self.vertices[i]].getPoint()
            side2 = start-vertices[self.vertices[i-1]].getPoint()
            cross = np.cross(side1,side2)
            if cross==0:
                #print("on edge")
                return True
        #print("poly[",self.index,"]")
        for i in range(0,len(self.vertices)):
            vector = (vertices[self.vertices[i]].getPoint()+vertices[self.vertices[i-1]].getPoint())/2
            vector = vector - start
            vector = vector/np.linalg.norm(vector)
            result = -1
            #print("interesetTest")
            #print("start:",start)
            #print("vector:",vector)
            for j in range(0,len(self.vertices)):
                edge  = np.array([vertices[self.vertices[j]].getPoint(),vertices[self.vertices[j-1]].getPoint()])
                #print("edge:",edge)
                if isInterset(start,vector,edge):
                    #print("interset")
                    result*=-1
            if result==-1:
                return False
        return True
    #find if a ray in the polygon. this is for when the starting point is on the edge, checking if the ray towards the polygon or not.
    def isHavingRay(self,point,vector,vertices):
        if not self.isHavingPoint(point,vertices):
            return False
        for i in range(0,len(self.vertices)):
            result = -1
            for j in range(0,len(self.vertices)):
                edge  = np.array([vertices[self.vertices[j]].getPoint(),vertices[self.vertices[j-1]].getPoint()])
                if isInterset(start,vector,edge):
                    result*=-1
            if result==-1:
                return False
        return True
    #get the array of vertex indices of an edge hold by the polygon. 
    #the vertex indices must be turned into coordinate format by meshObj before it is used for calculation. See Mesh.getPoints()
    def getEdge(self,i):
        edge1 = self.vertices[i]
        edge2 = self.vertices[(i+1)%len(self.vertices)]
        #print("vertexIndex:",edge1,",",edge2)
        return [edge1,edge2]
            
        
class Mesh:
    def __init__(self):
        self.name = " "
        self.vertices = []
        self.polygons = [] 
        self.scale = 1
        self.negative_x = 0
        self.negative_y = 0
        self.maxWeight = 0
        self.min_x = float("inf")
        self.min_y = float("inf")
        self.max_x = -float("inf")
        self.max_y = -float("inf")
    # find which polygon the point is in, return the index list of the polygons
    def findWhereisPoint(self,point):
        if point[0]>self.max_x:
            return []
        if point[1]>self.max_y:
            return []
        if point[0]<self.min_x:
            return []
        if point[1]<self.min_y:
            return []
        size = len(self.polygons)
        result =[]
        for index in range(0,size):
            if self.polygons[index].isHavingPoint(point,self.vertices):
                result.append(index)
        return result
    # find which polygon the ray is in , return the index of this polygon
    def findWhereisRay(self,point,vector):
        if point[0]>self.max_x:
            return []
        if point[1]>self.max_y:
            return []
        if point[0]<self.min_x:
            return []
        if point[1]<self.min_y:
            return []
        size = len(self.polygons)
        for index in range(0,size):
            if self.polygons[index].isHavingRay(point,vector,self.vertices):
                return index
        return -1
    #return the edge array passed from polygon into coordinate format
    def getPoints(self,indexs):
        result = []
        for i in indexs:
            result.append(self.vertices[i].getPoint())
        return np.array(result)
    #get the edge from polygon by polygon index and edge index
    #return: [[x1,y1],[x2,y2]]
    def getEdgefromPoly(self,polyIndex,edgeIndex):
        return self.getPoints(self.polygons[polyIndex].getEdge(edgeIndex))

In [8]:
import os
import cv2
import numpy as np

def readMesh(path):
    with open(path,'r',encoding = 'utf-8') as f:
        lines = f.readlines()
        n_vertices = 0
        n_poly = 0
        Type = lines[1].strip()
        n_vertices, n_poly = lines[2].split(" ")
        n_vertices= int(n_vertices)
        n_poly = int(n_poly)
        newMesh = Mesh()
        newMesh.name = path
        for i in range(3,3+n_vertices):
            line = lines[i]
            line = line.replace("\t"," ")
            l_vertice = line.split(" ")
            l_vertice = [j for j in l_vertice if(len(str(j))!=0)]
            newVertice = Vertex()
            newVertice.x=float(l_vertice[0])
            newVertice.y = float(l_vertice[1])
            newVertice.index = i-3
            
            num_nei = int(l_vertice[2])
            for j in range(3,3+num_nei):
                newVertice.neighbors.append(int(l_vertice[j]))
            newMesh.vertices.append(newVertice)

        for i in range(3+n_vertices, len(lines)):
            line = lines[i]
            line = line.replace("\t"," ")
            l_poly = line.split(" ")
            l_poly = [j for j in l_poly if(len(str(j))!=0)]
            newPoly = Polygon()
            n_ver = int(l_poly[0])
            newPoly.weight = int(l_poly[-1])
            del l_poly[-1]
            if newPoly.weight> newMesh.maxWeight:
                newMesh.maxWeight = newPoly.weight
            newPoly.index = i-3-n_vertices
            points = []
            for index in range(1,n_ver+1):
                vertice = int(l_poly[index])
                newPoly.vertices.append(vertice)
                verticeObj = newMesh.vertices[vertice]
                if verticeObj.x> newPoly.max_x:
                    newPoly.max_x = verticeObj.x
                if verticeObj.y> newPoly.max_y:
                    newPoly.max_y = verticeObj.y
                if verticeObj.x< newPoly.min_x:
                    newPoly.min_x = verticeObj.x
                if verticeObj.y< newPoly.min_y:
                    newPoly.min_y = verticeObj.y

            for index in range(n_ver+1,len(l_poly)):
                newPoly.neighbors.append(int(l_poly[index]))
            if Type == '2':
                newPoly.neighbors.append(newPoly.neighbors.pop(0))
            newMesh.polygons.append(newPoly)
            if newMesh.max_x< newPoly.max_x:
                newMesh.max_x = newPoly.max_x
            if newMesh.max_y< newPoly.max_y:
                newMesh.max_y = newPoly.max_y
            if newMesh.min_x> newPoly.min_x:
                newMesh.min_x = newPoly.min_x
            if newMesh.min_y> newPoly.min_y:
                newMesh.min_y = newPoly.min_y


    return newMesh


In [9]:
def printMesh(meshObj):
    w_path = meshObj.name.replace(".mesh",".png")
    size_y = 500
    size_x = 500
   
    scale_x = 1
    scale_y = 1

    
    max_x=0
    max_y=0
    
    negative_x = 0
    negative_y = 0
    
    for vertex in meshObj.vertices:
        if max_x<abs(vertex.x):
            max_x = abs(vertex.x)
        if max_y<abs(vertex.y):
            max_y = abs(vertex.y)
        if vertex.x<0 and vertex.x<negative_x:
            negative_x=vertex.x
        if vertex.y<0 and vertex.y<negative_y:
            negative_y = vertex.y
    
    negative_x-=max_x*0.1
    negative_y-=max_y*0.1
    
    #make the picture big enough to put the mesh
    while size_x <= max_x-negative_x:
        size_x*=2
    while size_y <= max_y-negative_y:
        size_y*=2
    #make the picture scalar big enough to fill the mesh
    while size_x>=2*(max_x-negative_x)*scale_x:
        scale_x*=2
    while size_y>=2*(max_y-negative_y)*scale_y:
        scale_y*=2
    meshObj.scale =max(scale_x,scale_y)
    #Scalar may over the size, so re fix it.
    while size_x <= (max_x-negative_x)*meshObj.scale:
        size_x*=2
    while size_y <= (max_y-negative_y)*meshObj.scale:
        size_y*=2
    
    
    meshObj.negative_x=negative_x
    meshObj.negative_y=negative_y
        
    canvas = np.ones((size_y,size_x , 3), dtype="uint8")
    canvas *= 255

    for poly in meshObj.polygons:
        points = []
        for i_vertex in poly.vertices:
            vertex = meshObj.vertices[i_vertex]
            x = vertex.x
            y = vertex.y
            x = (x-negative_x)*meshObj.scale
            y = (y-negative_y)*meshObj.scale
            points.append([x,y])  

        points = np.array(points, np.int32)
        points = points.reshape((-1,1,2))
        polyColor = (0,255*(1-poly.weight/meshObj.maxWeight),0)
        #cv2.polylines(canvas, pts=[points], isClosed=True, color=polyColor, thickness=1) 
        cv2.fillPoly(canvas, pts=[points], color=polyColor) 
    print("print mesh",w_path)
    return w_path,canvas
    

def printMeshes(projectbase, weightedFolder):
    for curDir, dirs, files in os.walk(projectbase+weightedFolder):
        for filename in files:
            if '.mesh' in filename:   
                r_path = curDir+'\\'+filename
                meshObj = readMesh(r_path)
                w_path,canvas = printMesh(meshObj)
                cv2.imwrite(w_path,canvas)

In [10]:
printMeshes(projectbase,weightedFolder)

print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\\arena-merged.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\\arena.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\\aurora-merged.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\\aurora.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-ambig.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear2-rot0.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear2-rot45.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear2-rot90.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\hard.png
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\square.png


In [11]:
def printPathesOnMap(pathes,meshObj):
    w_path,canvas = printMesh(meshObj)
    for path in pathes:
        for i in range(0,len(path)-1):
            
            start =path[i]
            end = path[i+1]
            start_x=int((start[0]-meshObj.negative_x)*meshObj.scale)
            start_y = int((start[1]-meshObj.negative_y)*meshObj.scale)
            end_x=int((end[0]-meshObj.negative_x)*meshObj.scale)
            end_y = int((end[1]-meshObj.negative_y)*meshObj.scale)
            print("from",start, "to",end)
            cv2.line(canvas, tuple([start_x,start_y]),tuple([end_x,end_y]), color=(0,0,255), thickness=1) 
    filePath = meshObj.name.replace(".mesh",".png")
    cv2.imwrite(filePath,canvas)
    return filePath

In [12]:
square = readMesh(squarePath)

In [13]:
start=np.array([4,3])
vector = np.array([1,-1])
polyIndex = square.findWhereisPoint(start)[0]
print(polyIndex)


1


In [4]:
#calculate the snellpath, return the direction vector of out ray [x1,y1]
#Edge[[x1,y1],[x2,y2]], O:inciPoint[x1,y1], start:startingPoint of inciVector[x1,y1]
def calSnellPath(Edge, O , weight1, weight2, start):
    v_Edge = np.array(Edge[1])-np.array(Edge[0])
    vn_Edge = v_Edge/np.linalg.norm(v_Edge)
    N = np.array([vn_Edge[1],-vn_Edge[0]])
    L = np.array(O)-np.array(start)
    L = L/np.linalg.norm(L)
    cos1 = np.dot(L,N)
    
    # the angle between inci vector and normal vector should be larger than 90degree, otherwise the normal vector is in opposite direction
    if cos1<0:
        cos1 *= -1
    else:
        N = -N
    weighteRate= weight1/weight2
    sin1_2 = (1-cos1**2)
    sin2_2 = sin1_2*(weighteRate**2)
    cos2_2 = 1 - sin2_2
    if sin2_2>1:
        sin1_2 = np.cross(L,N)
        return sin1_2,"refraction2Large"
    #print(sin2)
    out =  - N*(cos2_2**(0.5)) + weighteRate*(L+N*cos1)
    return out,"Done"

In [14]:
#throw snellRay from a polygon to next polygon.
#start:[x1,y1]the starting point inside the starting polygon
#polyIndex:int the index of starting polygon
#point:[x1,y1] the point which on the edge between starting polygon and target polygon
def throwSnellRay(meshObj,polyIndex,start,point):
    path = []
    #print("")
    #print("current poly",polyIndex)
    #print("throwing")
    
    #print("")
    
    #print("inci point ",point)  
    #print("start",start) 
    #print("polyIndex ",polyIndex)
    neibourIndex = meshObj.polygons[polyIndex].findEdgebyPoint(point,meshObj)
    if neibourIndex==-1:
        #point is not on the edge
        print("point:",point," not on the edge")
        return 
    #print("neibourIndex ",neibourIndex)
    #print("vector",vector)
    #print("point",point)
    path.append(point)
    nextPoly = meshObj.polygons[polyIndex].neighbors[neibourIndex]
    #print("nextPoly ",nextPoly)
    #print(meshObj.polygons[polyIndex].neighbors)
    out=None
    if nextPoly !=-1:
        weight1 = meshObj.polygons[polyIndex].weight
        #print("weight1",weight1)
        weight2 = meshObj.polygons[nextPoly].weight
        #print("weight2",weight2)
        edge = meshObj.getEdgefromPoly(polyIndex,neibourIndex)
        #print(edge)
        out,message = calSnellPath(edge,point,weight1,weight2,start)
        if message =="refraction2Large":
            return path,nextPoly,out,"refraction2Large"
        #print(start)
        #print(point)
        inciVector = point-start
        inciVector/=np.linalg.norm(inciVector)
        #print("inciVector",inciVector)
        #print("out",out)
        
        point,neibourIndex = meshObj.polygons[nextPoly].findIncidence(point,out,meshObj)
        #print("out point,",point)
        path.append(point)
        curPoly= nextPoly
    else:
        #print("NoMorePoly")
        #rint("start",start)
        #print("edge",edge)
        #print(point)
        #print("out",out)
        return path,nextPoly,out,"NoMorePoly"
    return path,nextPoly,out,"Done"
#throw snellRay passing multi polygons
def throwSnellRayThroughPolys(meshObj,polysequence,start,point):
    snellPath,nextPoly,out,message = throwSnellRay(meshObj,polysequence[0],start,point)
    if message=="NoMorePoly":
        return snellPath,nextPoly,out,message
    #print("snellPath",snellPath)
    for i in range(1,len(polysequence)):
        path,nextPoly,out,message = throwSnellRay(meshObj,polysequence[i],snellPath[-2],snellPath[-1])
        #print("path",path)
        if message=="NoMorePoly":
            return snellPath,nextPoly,out,message
        snellPath.append(path[1])
    return snellPath,nextPoly,out,message

In [26]:
#print(square.polygons[3].neighbors)
#print(start)
#print(vector)
print(polysequence)
newPoint = (newInterval[0]+newInterval[1])/2
print(newPoint)
snellPath,nextPoly,out,message=throwSnellRayThroughPolys(square,[1,3],start,newPoint)
print(snellPath)
#path,nextPoly,out = throwSnellRay(square,3,snellPath[0],snellPath[1])
#throwSnellRay(square,3,np.array([4.61425781, 2.        ]),np.array([7.40672913, 0.59327087]))
#print(path)                                          


[1, 3, 5]
[4.61453247 2.        ]
[array([4.61453247, 2.        ]), array([7.40491152, 0.59508848]), array([9.        , 0.65894223])]


In [15]:
#find the snellpath reach the target point through a seuqence of polygon
def findSnellPathToPoint(meshObj,start,polysequence,interval,target):
    distance = float("inf")
    middle = (interval[0]+interval[1])/2
    path = None
    polySequence = None
    #if interval[1] is on right side
    right = interval[1]
    left = interval[0]
    if np.cross(middle-start,right-start)<0:
        #print("npcross: ",np.cross(middle-start,right-start))
        tmp = right
        right = left
        right = tmp
    #while distance<np.linalg.norm(interval)/10:
    for i in range(0,20):
        #print("")
        #print("middle ",middle)
        #print("left ",left)
        #print("right ",right)
        vector = middle-start
        #print("vector ",vector)
        #vector = vector/np.linalg.norm(vector)
        #isContinue=True
        
        #print("polyIndex",polyIndex)
        #print("start",start)
        #print("middle",middle)
        #print("polysequence",polysequence)
        path,polySequence,out,message= throwSnellRayThroughPolys(meshObj,polysequence,start,middle)
        if message == "Done":
            distance = np.linalg.norm(path[-1]-target)
            d_m2p = np.linalg.norm(path[-1]-middle)
            d_m2t = np.linalg.norm(target-middle)
            #print("distance ",distance)
            #print("path ",path)
            if distance<approximate and d_m2p<d_m2t :
                #print("distance is small enough")
                path[-1]=target
                return path,polySequence
            # at right
            if np.cross(target-path[-2],path[-1]-path[-2])<0:
                #print("too left")
                left=middle
                middle = (left+right)/2
            #at left
            else:
                #print("too right")
                right = middle
                middle = (left+right)/2
        else:
            if message == "refraction2Large":
                #isContinue = False
                sin1 = out
                #print("sin too large ",sin1)
                #print("dot",dot)
                if sin1>0:
                    right = middle
                    middle = (right+left)/2  
                else:
                    left = middle
                    middle = (right+left)/2  
            if message == "NoMorePoly":
                #print("NoMorePoly")
                #print("NomorePoly")
                #print("path",path)
                
                #need modification
                if np.cross(target-middle,path[-1]-middle)<0:
                    #print("too left")
                    right=middle
                    middle = (left+right)/2
                #at left
                else:
                    #print("too right")
                    left = middle
                    middle = (left+right)/2
    return path,polySequence
            

            
        
        

In [17]:
print(polyIndex)
print(target)
snellPath,polysequence = findSnellPathToPoint(square,start,[polyIndex],interval,target)
polysequence = [polyIndex,polysequence]
print(snellPath)
print(polysequence)

1
[7.65681565 0.34318435]
[array([4.57989502, 2.        ]), array([7.65681565, 0.34318435])]
[1, 3]


In [42]:
findSnellPathToPoint(square,start.copy(),polysequence.copy(),newInterval.copy(),[9,0.4])

([array([4.68297616, 2.        ]),
  array([7.02088608, 0.97911392]),
  array([7.38110764, 1.        ])],
 -1)

In [18]:
def calCost(path,meshObj,p_sequence):
    cost = 0      
    #print("polysequence:",p_sequence)
    for i in range(0,len(p_sequence)):
        #print("section in poly:",p_sequence[i])
        tmp = path[i+1]-path[i]
        tmp = np.linalg.norm(tmp)
        sectionCost = tmp *meshObj.polygons[p_sequence[i]].weight
        #print("sectionCost:",sectionCost)
        cost+=sectionCost
    return cost

In [27]:
def printSnellPath(meshObj,snellPath, start):
    pathes = []
    pathes.append(start)
    for i in range(0,len(snellPath)-1):
        pathes.extend([snellPath[i],snellPath[i+1]])
    print(pathes)
    pathes = [pathes]
    filePath = printPathesOnMap(pathes,meshObj)
printSnellPath(square,snellPath, start)

[array([3.5, 3. ]), array([4.61453247, 2.        ]), array([7.40491152, 0.59508848]), array([7.40491152, 0.59508848]), array([9.        , 0.65894223])]
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear.png
from [3.5 3. ] to [4.61453247 2.        ]
from [4.61453247 2.        ] to [7.40491152 0.59508848]
from [7.40491152 0.59508848] to [7.40491152 0.59508848]
from [7.40491152 0.59508848] to [9.         0.65894223]


In [28]:
square = readMesh(squarePath)

find a taret but we don't know where should we exist. Critical angle.\
implement a search function. Node, seccessor,.\
heustic.

In [29]:
collinearPath = "F:\\ANU\\Research\\Polyanya\\anyangle\\polyanya\\weightedMeshes\\tests\\bad-collinear2-rot0.mesh"
collinear = readMesh(collinearPath)

In [30]:
#polygon 3 in collinear2 weight 9 while polygon 6 in collinear2 weight 4.
print("polygon 3:")
print(collinear.getPoints(collinear.polygons[3].vertices))
print("weight:",collinear.polygons[3].weight)
print("polygon 6:")
print(collinear.getPoints(collinear.polygons[6].vertices))
print("weight:",collinear.polygons[6].weight)
#polygon 3 next to polygon 6.
#Suppose there is a starting point in polygon 3, the path walking on the edge between polygon 3 and 6, and back to polygon 3 again.
#This path might still be shorter than a stiaght line in polygon3 because the weight in polygon 6 is far small than it is in polygon 3.

polygon 3:
[[5. 1.]
 [6. 1.]
 [7. 1.]
 [7. 2.]
 [4. 2.]
 [4. 1.]]
weight: 9
polygon 6:
[[7. 2.]
 [7. 3.]
 [4. 3.]
 [4. 2.]]
weight: 4


In [31]:
start=np.array([6,1.5])
target = np.array([4,1.5])
Interval = np.array([[7,2],[4,2]])
snellPaths3,nextPolyID = findSnellPathToPoint(collinear,start,[3],Interval,[4,2])
print(snellPaths3)
print(nextPolyID)
printSnellPath(collinear,snellPaths3,start)
snellPaths3[1][1] = 2

[array([5.7519331, 2.       ]), array([4.        , 2.00713966])]
6
[array([6. , 1.5]), array([5.7519331, 2.       ]), array([4.        , 2.00713966])]
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear2-rot0.png
from [6.  1.5] to [5.7519331 2.       ]
from [5.7519331 2.       ] to [4.         2.00713966]


In [32]:
def findPathToPointWalkingOnInterval(meshObj,interval,polySequence,target):
    #print(sequence)
    snellPaths,nextPoly=findSnellPathToPoint(collinear,target,polySequence,interval,interval[0])
    path =[snellPaths[0],target]
    return path,polyIndex

In [33]:
print(snellPaths3)
snellPaths4, nextPoly4 = findPathToPointWalkingOnInterval(collinear,snellPaths3,[3],target)
print("snellPaths4",snellPaths4)
print("")

snellPaths34 =[]
snellPaths34.append(snellPaths3[0])
snellPaths34.append(snellPaths4[0])
snellPaths34.append(snellPaths4[1])

print("snellPaths34",snellPaths34)
print("")

printSnellPath(collinear,snellPaths34,start)

[array([5.7519331, 2.       ]), array([4., 2.])]
snellPaths4 [array([4.24806811, 2.        ]), array([4. , 1.5])]

snellPaths34 [array([5.7519331, 2.       ]), array([4.24806811, 2.        ]), array([4. , 1.5])]

[array([6. , 1.5]), array([5.7519331, 2.       ]), array([4.24806811, 2.        ]), array([4.24806811, 2.        ]), array([4. , 1.5])]
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear2-rot0.png
from [6.  1.5] to [5.7519331 2.       ]
from [5.7519331 2.       ] to [4.24806811 2.        ]
from [4.24806811 2.        ] to [4.24806811 2.        ]
from [4.24806811 2.        ] to [4.  1.5]


In [34]:
def findPathToPointWalkingOnIntervalwithPrevisousInci(meshObj,interval,target,start):   
    v_Edge = np.array(interval[1])-np.array(interval[0])
    vn_Edge = v_Edge/np.linalg.norm(v_Edge)
    N = np.array([vn_Edge[1],-vn_Edge[0]])
    L = np.array(interval[0])-np.array(start)
    L = L/np.linalg.norm(L)
    cos1 = np.dot(L,N)
    # the angle between inci vector and normal vector should be larger than 90degree, otherwise the normal vector is in opposite direction
    if cos1<0:
        cos1 *= -1
    else:
        N = -N
    L2 = (L+N*cos1)
    L1 = -(L-L2)
    #print(sin2)
    out =  L1+L2
    print(-out)
    #if isInterset(target,-out,interval):
    intersetPoint = getIntersetPoint(target,-out,interval)
    path = [intersetPoint,target]
    return path
    #else:
    #    return None

In [35]:
snellPaths4 = findPathToPointWalkingOnIntervalwithPrevisousInci(collinear,snellPaths3,target,start)
print("snellPaths4",snellPaths4)
print("")

snellPaths34 =[]
snellPaths34.append(snellPaths3[0])
snellPaths34.append(snellPaths4[0])
snellPaths34.append(snellPaths4[1])

print("snellPaths34",snellPaths34)
print("")

printSnellPath(collinear,snellPaths34,start)

[0.44444075 0.89580825]
snellPaths4 [array([4.2480669, 2.       ]), array([4. , 1.5])]

snellPaths34 [array([5.7519331, 2.       ]), array([4.2480669, 2.       ]), array([4. , 1.5])]

[array([6. , 1.5]), array([5.7519331, 2.       ]), array([4.2480669, 2.       ]), array([4.2480669, 2.       ]), array([4. , 1.5])]
print mesh F:\ANU\Research\Polyanya\anyangle\polyanya\weightedMeshes\tests\bad-collinear2-rot0.png
from [6.  1.5] to [5.7519331 2.       ]
from [5.7519331 2.       ] to [4.2480669 2.       ]
from [4.2480669 2.       ] to [4.2480669 2.       ]
from [4.2480669 2.       ] to [4.  1.5]


In [36]:
class Node:
    def __init__(self,meshObj):
        self.meshObj = meshObj
        self.parentNode = None
        #record the cost for reaching this node from root
        self.cost = 0
    def getSuccesors(self):
        return None
    def initial(self):
        return None
    def isHavingTarget(self,polyList,point):
        return False
    def isTarget(self,point):
        return False
    def getTotalCost(self):
        if self.parentNode!=None:
            return self.cost+self.parentNode.getTotalCost()
        else:
            return self.cost

class ConeNode(Node):
    def __init__(self,meshObj):
        super(ConeNode,self).__init__(meshObj)
        self.type = 'Cone'
        self.startPoint = []
        self.curInterval = []
        self.targetInterval =[]
        self.polySequence = []
        self.curPoly = -1
    def show(self):
        print("------------------------------------")
        print(self.type)
        print("startPoint:",self.startPoint)
        print("curInterval:",self.curInterval)
        print("targetInterval:",self.targetInterval)
        print("polySequence:",self.polySequence)
        print("curPoly:",self.curPoly)
        print("------------------------------------")
    def calCost(self):
        ##need to write something here!!!
        #the smellest path in cone
        return False
    #if point is in the last polygon in self.polySequence, then return True
    def isHavingTarget(self,polyList,point):
        #need to check if the tartget is precisely in the cone
        for i in polyList:
            if i == self.polySequence[-1]:
                return True
    #Should only be called if isHavingTarget is True
    def getPath(self,point):
        if len(self.polySequence)==1:
            selfPath = [self.startPoint,point]
        else:
            selfPath,tmpsequence = findSnellPathToPoint(self.meshObj,self.startPoint.copy(),self.polySequence,self.curInterval,point)
        totalPath = []
        #if it has parentNode, get path from parent node
        if self.parentNode!=None:
            totalPath.extend(self.parentNode.getPath(self.startPoint))
        else:
            totalPath.append(self.startPoint)
        totalPath.extend(selfPath)
        return totalPath
         

    def getSuccesors(self,state):
        snellPaths=[]
        succesors = []
        
        polyList,target = state
        #if there is a target in the last cone , create a cornner point node from starting point to target
        if self.isHavingTarget(polyList,target):
            print("having target in the cone!")
            if len(self.polySequence)!=1:
                newPath,tmpsequence = findSnellPathToPoint(self.meshObj,self.startPoint,self.polySequence,self.curInterval,target)
                if newPath!=None:
                    succesor = CornerPointNode(self.meshObj)
                    succesor.type = 'Corner'
                    del newPath[-1]
                    newPath = [self.startPoint].extend(newPath)
                    succesor.prePath = newPath
                    succesor.polySequence=self.polySequence
                    succesor.curPoly = self.curPoly
                    succesor.cornerPoint = target
                    succesor.parentNode = self
                    succesors.append(succesor)
                else:
                    newPath = [self.startPoint]
                    succesor = CornerPointNode(self.meshObj)
                    succesor.type = 'Corner'
                    succesor.prePath = newPath
                    succesor.polySequence=self.polySequence
                    succesor.curPoly = self.curPoly
                    succesor.cornerPoint = target
                    succesor.parentNode = self
                    succesors.append(succesor)
            
        #do not execute following code if there is no more poly after passing the current interval
        if self.curPoly==-1:
            return succesors
     
        
        curPolyObj = self.meshObj.polygons[self.curPoly]
        size = len(curPolyObj.vertices)
        for i in range(size):
            #print("startPoint",self.startPoint)
            #print("polySequence",self.polySequence)
            #print("curInterval",self.curInterval)
            #print("target",curPolyObj.vertices[i])
            newPath,tmpsequence = findSnellPathToPoint(self.meshObj,self.startPoint,self.polySequence,self.curInterval,
                                                       curPolyObj.vertices[i])
            
            snellPaths.append(newPath)
        #print("snellPaths:",snellPaths)
        for i in range(size):
            nextNeighborIndex = curPolyObj.neighbors[i]
            #print("nextNeighborIndex:",nextNeighborIndex)
            #nextNeighborIndex =-1 means there is no more poly. self.polySequence[-1] is the polygon where the node is in
            if nextNeighborIndex !=-1 and nextNeighborIndex!=self.polySequence[-1]:
                succesor = ConeNode(self.meshObj)
                succesor.curPoly = nextNeighborIndex
                succesor.startPoint = self.startPoint
                succesor.polySequence = self.polySequence.copy()
                succesor.polySequence.append(self.curPoly)
                succesor.type = 'Cone'
                succesor.curInterval = [snellPaths[i][0],snellPaths[(i+1)%size][0]]
                succesor.targetInterval = [snellPaths[i][-1],snellPaths[(i+1)%size][-1]]
                succesor.parentNode = self
                if not (snellPaths[i][-1]==curPolyObj.vertices[i]).all():
                    ## if two succesor next to each other go to different polygons, we need to create a corner point between them(on the vertex) 
                    ##"some point cannot be reach!!"" The weight of next poly is larger than current poly!! 
                    ##might need to create cornerPoint Successor!!!
                    ##unSolve!!
                    Sin2Large = True
                succesors.append(succesor)
        for i in range(size):
            if curPolyObj.neighbors[i]==self.polySequence[-1]:
                if (snellPaths[i][-1] == curPolyObj.vertices[i]).all():
                    ##we found a critical angle here!!!
                    ##need to create walk on edge node !!!
                    ##unSolve!!
                    criticalPoint = True
                if (snellPaths[(i+1)%size][-1] == curPolyObj.vertices[(i+1)%size]).all():
                    ##we found a critical angle here!!!
                    ##need to create walk on edge node !!!
                    ##unSolve!!
                    criticalPoint = True
        return succesors

            
        

class walkOnEdgeNode(Node):
    def __init__(self,meshObj):
        super(walkOnEdgeNode,self).__init__(meshObj)
        self.type = 'Edge'
        #walk starting from curInterval[0] to curInterval[1]
        self.curInterval = []
        #it represents
        self.targetInterval = []
        #the index of poly where contribute the weight for walking on edge
        self.startPoly = -1
        #the sequence of poly where the ray pass after exist edge                                           
        self.polySequence = []
        self.curPoly = -1
    def show(self):
        print("------------------------------------")
        print(self.type)
        print("startPoly:",self.startPoly)
        print("curInterval:",self.curInterval)
        print("targetInterval:",self.targetInterval)
        print("polySequence:",self.polySequence)
        print("curPoly:",self.curPoly)
        print("------------------------------------")
    def calCost(self):
        ##need to write something here!!!
        return False
    def isHavingTarget(self,polyList,point):
        for i in polyList:
            if i == self.curPoly:
                return True
    #Should only be called if isHavingTarget is True
    def getPath(self,point):
        if len(self.polySequence)==1:
            selfPath = [self.startPoint,point]
        else:
            ##need a new find Snell path point here because its different to cone node
            selfPath,tmpsequence = findSnellPathToPoint(self.meshObj,self.startPoint,self.polySequence,self.curInterval,point)
        totalPath = []
        #if it has parentNode, get path from parent node
        if self.parentNode!=None:
            totalPath.extend(self.parentNode.getPath(self.startPoint))
        else:
            totalPath.append(self.startPoint)
        totalPath.extend(selfPath)
        return totalPath
    def getSuccesors(self,state):
        curPolyObj = self.meshObj.polygons[self.curPoly]
        
        
        
class CornerPointNode(Node):
    def __init__(self,meshObj):
        super(CornerPointNode,self).__init__(meshObj)
        self.type = 'Corner'
        #previous path, not include cornerPoint itself
        self.prePath = []
        self.polySequence=[]
        self.curPoly = -1
        self.cornerPoint = []
    def show(self):
        print("------------------------------------")
        print(self.type)
        print("cornerPoint:",self.cornerPoint)
        print("prePath:",self.prePath)
        print("polySequence:",self.polySequence)
        print("curPoly:",self.curPoly)
        print("------------------------------------")
    def calCost(self):
        ##need to write something here!!!
        self.cost = calCost(self.prePath.copy().append(self.cornerPoint), self.meshObj, self.polySequence.copy().append(self.curPoly))
    def isTarget(self,point):
        if (point == self.cornerPoint).all():
            return True
    #should only be called if isHavingTarget(self,polyList,point) return True
    def getPath(self,point):
        selfPath = self.prePath[1:]
        selfPath.append(self.cornerPoint)
        totalPath = []
        #if it has parentNode, get path from parent node
        if self.parentNode!=None:
            #if it has prePath, use the starting point of prePath as the target for parentNode
            if len(self.prePath) !=0:
                totalPath.extend(self.parentNode.getPath(self.polySequence[0]))
            else:
                totalPath.extend(self.parentNode.getPath(self.cornerPoint))
        else:
            if len(self.prePath) !=0:
                totalPath.append(self.prePath[0])
            else:
                totalPath.append(self.cornerPoint)
        totalPath.extend(selfPath)
        return totalPath
    def getSuccesors(self,state):
        #write code here
        curPolyObj = self.meshObj.polygons[self.curPoly]
        PathsToVertices=[]
        succesors = []
        size = len(curPolyObj.vertices)
        for i in range(size):
            nextNeighborIndex = curPolyObj.neighbors[i]
            #nextNeighborIndex =-1 means there is no more poly. self.polySequence[-1] is the polygon where the node is in
            if len(self.polySequence)==0 or nextNeighborIndex!=self.polySequence[-1]:
                succesor = ConeNode(self.meshObj)
                succesor.curPoly = nextNeighborIndex
                succesor.startPoint = self.cornerPoint.copy()
                succesor.polySequence = [self.curPoly]
                succesor.type = 'Cone'
                succesor.curInterval = [curPolyObj.vertices[i],curPolyObj.vertices[(i+1)%size]]
                succesor.curInterval = self.meshObj.getPoints(succesor.curInterval)
                succesor.targetInterval = succesor.curInterval
                succesors.append(succesor)
                #print("showSucce:")
                #print("--------------------")
                #succesor.show()
                #print("--------------------")

        return succesors

        

In [44]:

import queue


def startSearch(meshObj,start,point):
    startingNode = CornerPointNode(meshObj)
    vector = (point-start)
    vector/=np.linalg.norm(vector)
    startingNode.curPoly = meshObj.findWhereisRay(start,vector)
    startingNode.cornerPoint = start
    targetPolyList = meshObj.findWhereisPoint(point)
    state = targetPolyList,point  
    print("targetPolyList:",targetPolyList)
    NodeQueue = queue.Queue()
    NodeQueue.put(startingNode)

    while not NodeQueue.empty():
        node = NodeQueue.get()
        node.show()
        if node.isTarget(point):
            print("found Target!!!")
            return node
        succesors = node.getSuccesors(state)
        print("get ",len(succesors))
        for succesor in succesors:
            NodeQueue.put(succesor)

In [43]:
start=np.array([3.5,3])
target = newPoint
print("target",target)
startSearch(square,start,target)

target [4.61453247 2.        ]
targetPolyList: [1, 3]
------------------------------------
Corner
cornerPoint: [3.5 3. ]
prePath: []
polySequence: []
curPoly: 1
------------------------------------
get  6
------------------------------------
Cone
startPoint: [3.5 3. ]
curInterval: [[1. 2.]
 [4. 2.]]
targetInterval: [[1. 2.]
 [4. 2.]]
polySequence: [1]
curPoly: -1
------------------------------------
having target in the cone!
get  0
------------------------------------
Cone
startPoint: [3.5 3. ]
curInterval: [[4. 2.]
 [6. 2.]]
targetInterval: [[4. 2.]
 [6. 2.]]
polySequence: [1]
curPoly: 3
------------------------------------
having target in the cone!
get  2
------------------------------------
Cone
startPoint: [3.5 3. ]
curInterval: [[6. 2.]
 [9. 2.]]
targetInterval: [[6. 2.]
 [9. 2.]]
polySequence: [1]
curPoly: -1
------------------------------------
having target in the cone!
get  0
------------------------------------
Cone
startPoint: [3.5 3. ]
curInterval: [[ 9.  2.]
 [10.  3.]]


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
for poly in square.polygons:
    print(poly.vertices)
    print(poly.neighbors)
    print(poly.weight)
    print("-----------")
    